# Secbert를 위한 데이터 준비 & 크롤링
190715

In [1]:
import requests
import pandas as pd
import re
import time
from bs4 import BeautifulSoup
import string
import datetime
from datetime import datetime

In [2]:
save_output = 'security_archive_paper_39.txt'

In [3]:
# URL 작성
base_url = 'https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=security&terms-0-field=all&classification-computer_science=y&classification-eess=y&classification-physics_archives=all&classification-include_cross_list=include&date-year=&date-filter_by=date_range&date-from_date=2015&date-to_date=&date-date_type=submitted_date&abstracts=hide&size=50&order=-announced_date_first'
# 페이지 넘어가며 뒤에 붙는 주소
url_list = ['']
temp = list(range(50, 10000, 50))  # '', '&start=50', '&start=100', ...8950 -> 총 10,106개
for i in temp: url_list.append('&start={}'.format(i))

In [4]:
def printable_text_true(text):
    # printable 하면 False
    # 이상한게 있으면 True
    not_printable = 0
    for char in text:
        if char not in string.printable:
            not_printable += 1
    if not_printable == 0:
        return False
    else:
        return True

In [ ]:
file = open(save_output, 'w', encoding='UTF8')

start = time.clock()
count_doc, count_pass = 0, 0
for i in range(39, len(url_list)):
    print('%d / %d - %d / %d, %.19s' %(i, len(url_list), count_doc, count_pass, datetime.now()))
    
    tmp_url = base_url + url_list[i]  # 페이지 넘어가는 url 만듬
    
    ## 막히는 경우 예외처리
    tmp_response = ''
    while tmp_response == '':
        try:
            tmp_response = requests.get(tmp_url)
            break
        except:
            print("Connection refused, wait 5 sec")
            time.sleep(5)
            continue            
    
    tmp_bs = BeautifulSoup(tmp_response.text, 'html.parser')
    tmp_contents = tmp_bs.select('li.arxiv-result')
    tmp_list = [tmp_href.select_one('a').attrs.get('href') for tmp_href in tmp_contents]

    # ['https://arxiv.org/abs/1906.11488',
    #  'https://arxiv.org/abs/1906.11465', ...]

    for j, tmp_paper in enumerate(tmp_list):
        # tmp_paper : archive 주소!
        tmp_response_paper = requests.get(tmp_paper)  # <Response [200]>
        tmp_bs_paper = BeautifulSoup(tmp_response_paper.text, 'html.parser')

        title = re.sub('\\s+',
                         ' ',
                         tmp_bs_paper.select_one('h1.title.mathjax').text.replace('Title:', ''))
        abstract = tmp_bs_paper.select_one('blockquote.abstract.mathjax').text.replace('Abstract:  ', '').replace('\n', ' ')

        if '$' in abstract or '$' in title or printable_text_true(title) or printable_text_true(abstract):
            count_pass += 1
            pass
        # 다 printable 하면
        else:
            # 영어, 숫자, () 빼고 다 삭제
            title = re.sub('[^a-zA-Z1-9()% ]', '', title)
            abstract = re.sub('[^a-zA-Z1-9()% ]', '', abstract)
            
            file.write("<<doc>>\n")
            file.write(title+'\n')
            file.write(abstract+'\n\n')
            count_doc += 1

end = time.clock()
print('%d sec'%end - start)
print(count_doc)


39 / 200 - 0 / 0, 2019-07-15 17:48:27
40 / 200 - 43 / 7, 2019-07-15 17:49:14
41 / 200 - 90 / 10, 2019-07-15 17:50:03
